In [15]:
using CSV
using DataFrames
using DataStructures
using Distributions
using GZip
using JSON
using Logging
using ProgressMeter
using PyPlot
using PyCall

using Distributions
using Random

import Base.isless

In [69]:
X = 1:0.13:7
P = (X.^2)
P ./= sum(P)

dist = DiscreteNonParametric(X, P)

DiscreteNonParametric{Float64,Float64,StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}},Array{Float64,1}}(
support: 1.0:0.13:6.98
p: [0.0011180731038682085, 0.0014276675463293152, 0.0017750528597011682, 0.0021602290439837655, 0.002583196099177109, 0.0030439540252811975, 0.0035425028222960324, 0.004078842490221612, 0.004652973029057937, 0.005264894438805007  …  0.03774178750148563, 0.039449644167644335, 0.04119529170471376, 0.042978730112693944, 0.04479995939158486, 0.046658979541386536, 0.04855579056209895, 0.050490392453722104, 0.05246278521625601, 0.054472968849700674]
)


In [89]:
@enum FRUIT apple=1 orange

In [111]:
Categorical

DiscreteNonParametric{Int64,P,Base.OneTo{Int64},Ps} where Ps where P

In [83]:
X = [Severe, Critical]
P = [0.5, 0.5]
DiscreteNonParametric(X, P)

MethodError: MethodError: no method matching DiscreteNonParametric(::Array{Severity,1}, ::Array{Float64,1})
Closest candidates are:
  DiscreteNonParametric(!Matched::Ts, ::Ps; check_args) where {T<:Real, P<:Real, Ts<:AbstractArray{T,1}, Ps<:AbstractArray{P,1}} at /Users/tomek/.julia/packages/Distributions/dTXqn/src/univariate/discrete/discretenonparametric.jl:36

In [100]:
@time sample(X, 10^6);

  0.023631 seconds (3 allocations: 3.815 MiB)


In [110]:
@time rand(Gamma(1,1), 10^6);

  0.028416 seconds (2 allocations: 7.629 MiB)


In [103]:
@time rand(dist, 10^6);

  0.044631 seconds (9 allocations: 7.632 MiB)


In [5]:
df = GZip.open("data/simulations/wroclaw-population-orig.csv.gz","r") do io
    df = CSV.read(io, copycols=true)  # read CSV as DataFrame
    # create new DataFrame with
    DataFrame(
        age=Int8.(df.age),
        gender = df.gender .== 1,
        household_index = Int32.(df.household_index)
    )
end

,age,gender,household_index
,Int8,Bool,Int32
1,0,0,195338
2,0,0,262737
3,0,0,219588
4,0,0,133782
5,0,0,266169
6,0,0,131916
7,0,0,186665
8,0,0,268759
9,0,0,233547


In [2]:
@enum HealthState Healthy Infected Infectious StayingHome Hospitalized Recovered Dead

@enum Severity Asymptomatic=1 Mild Severe Critical

@enum ContactKind::UInt8 HouseholdContact FriendshipContact SporadicContact ConstantKernelContact

In [3]:
abstract type AbstractEvent end

abstract type DiseaseEvent <: AbstractEvent end
abstract type InfectionEvent <: AbstractEvent end

struct BecomeInfectiousEvent <: DiseaseEvent
    time::Float32
    subject_id::Int32
end

struct StayHomeEvent <: DiseaseEvent
    time::Float32
    subject_id::Int32
end

struct GoHospitalEvent <: DiseaseEvent
    time::Float32
    subject_id::Int32
end

struct DeathEvent <: DiseaseEvent
    time::Float32
    subject_id::Int32
end

struct RecoverEvent <: DiseaseEvent
    time::Float32
    subject_id::Int32
end

struct OutsideInfectionEvent <: InfectionEvent
    time::Float32
    subject_id::Int32
end

struct TransmissionEvent <: InfectionEvent
    time::Float32
    subject_id::Int32
    source_id::Int32
    kind::ContactKind
end


In [6]:
# the constant part of the simulation
struct SimParams 
    individual_df::DataFrame    # the sampled population
    household_df::DataFrame     # sampled households
    progression_df::DataFrame   # sampled progression times
   
    constant_kernel_param::Float64
    
    dist_severity::Categorical
    
    dist_incubation_time::DiscreteNonParametric
    dist_t0_to_t1::DiscreteNonParametric
    dist_t0_to_t2::DiscreteNonParametric
    dist_recovery::DiscreteNonParametric
    
    dist_death_prob::DiscreteNonParametric
    dist_death_time::DiscreteNonParametric
    
end

# the mutable part of the simulation
mutable struct SimState
    time::Float64
    queue::BinaryHeap{AbstractEvent} # TODO change to union once all events are implemented
    health_states::Vector{HealthState}
    detected::BitVector
    
    infections::SortedSet{InfectionEvent} # TODO change to union 
    
    num_dead::Int
    num_affected::Int
    num_detected::Int
end

In [13]:
function add_potential_contractions_from_constant_kernel!(state::SimState, params::SimParams, source_id::Integer)
    progession = params.progression_df[person_id]

    start_time = progressions.t0
    end_time = missing == progressions.t1 ? progressions.t2 : progressions.t1
        
    time_dist = Uniform(start_time, end_time)
    
    total_infection_rate = (end_time - start_time) * param.constant_kernel_param
    num_infections = random(Poisson(total_infection_rate))
    
    if num_infections == 0
        return
    end
    
    num_individuals = size(params.individual_df, 1)
    selected_individuals = sample(num_individuals-1, num_infecitons) # exclude the source itself
    
    for subject_id in selected_individuals
        if subject_id >= source_id # restore the indexing
            subject_id +=1 
        end
        
        if Healthy == subjecthealt(subject_id) 
            infection_time = rand(time_dist)
            push!(state.queue, TransmissionEvent(infection_time, source_id, subject_id, ConstantKernelContact))
        end
    end
end

add_potential_contractions_from_constant_kernel (generic function with 1 method)

In [ ]:
function generate_progression_times(params::SimParams, severity::Severity)
    incubation_time = sample(params.dist_incubation)
        
    
    if severity == Severe || severity == Critical    
        t2 = incubation_time + sample(params.dist_t0_to_t2)
    else
    t1 = t0 + sample(params.dist_t0_to_t1)
        
end

In [ ]:
def generate_disease_progression(self, person_id, event_time: float,
                                     initial_infection_status: InfectionStatus) -> None:
        """Returns list of disease progression events
        "future" disease_progression should be recalculated when the disease will be recognised at the state level
        t0 - time when individual becomes infectious (Mild symptoms)
        t1 - time when individual stay home/visit doctor due to Mild/Serious? symptoms
        t2 - time when individual goes to hospital due to Serious symptoms
        """
        if initial_infection_status == InfectionStatus.Contraction:
            tminus1 = event_time
            t0 = tminus1 + self.generate_random_sample(**self.disease_progression[T0])
            self.append_event(Event(t0, person_id, T0, person_id, DISEASE_PROGRESSION, tminus1))
        elif initial_infection_status == InfectionStatus.Infectious:
            t0 = event_time
            # tminus1 does not to be defined, but for completeness let's calculate it
            tminus1 = t0 - self.generate_random_sample(**self.disease_progression[T0])
        else:
            raise ValueError(f"invalid initial infection status {initial_infection_status}")
        t2 = None
        # t3 = None
        if self._expected_case_severity[person_id] in [
            ExpectedCaseSeverity.Severe,
            ExpectedCaseSeverity.Critical
        ]:
            t2 = t0 + self.generate_random_sample(**self.disease_progression[T2])
            self.append_event(Event(t2, person_id, T2, person_id, DISEASE_PROGRESSION, t0))

        t1 = t0 + self.generate_random_sample(**self.disease_progression[T1])
        if not t2 or t1 < t2:
            self.append_event(Event(t1, person_id, T1, person_id, DISEASE_PROGRESSION, t0))
        else:
            t1 = None
        #self._df_progression_times =
        tdetection = None
        trecovery = None
        tdeath = None
        if np.random.rand() <= self._params[DEATH_PROBABILITY][self._expected_case_severity[person_id].value]:
            tdeath = t0 + self.generate_random_sample(**self.disease_progression[TDEATH])
            self.append_event(Event(tdeath, person_id, TDEATH, person_id, DISEASE_PROGRESSION, t0))
        else:
            #trecovery =
            pass
        self._progression_times_dict[person_id] = {ID: person_id, TMINUS1: tminus1, T0: t0, T1: t1, T2: t2,
                                                   TDEATH: tdeath}
                                                   #TDETECTION: tdetection, TRECOVERY: trecovery, TDEATH: tdeath}
        if initial_infection_status == InfectionStatus.Infectious:
            self.handle_t0(person_id)

In [9]:
isactive(state::HealthState) = (state == Infectious) || (state == StayingHome)

isactive (generic function with 1 method)

In [7]:
time(event::AbstractEvent) = event.time 
isless(e1::AbstractEvent, e2::AbstractEvent) = time(e1) < time(e2)

subject(event::AbstractEvent) = event.subject_id
source(event::TransmissionEvent) = event.source_id

health(state::SimState, person_id::Integer) = state.infection_status[person_id]

subjecthealth(state::SimState, event::AbstractEvent) = health(state, subject(event))
sourcehealth(state::SimState, event::TransmissionEvent) = health(state, source(event))

sourcehealth (generic function with 1 method)

In [ ]:
function register_infection(state::SimState, params::SimParams, subject_id::Integer, source_id::Integer, kind::ContactKind)
    
end

In [12]:
function execute!(state::SimState, params::SimParams, event::OutsideInfectionEvent)
    @assert Recovered !== subjecthealth(state, event)
    if Healthy == subjecthealth(state, event)
        state.infection_status[event.person_id] = Infected
        push!(state.infections, event)
    end
end

function execute!(state::SimState, params::SimParams, event::TransmissionEvent)
    if Healthy != subjecthealth(state, event) || !isactive(sourcehealth(event))
        return
    end
    
    if event.kind != HouseholdContact
        if sourcehealth(event) != StayingHome 
            push!(state.infections, event)
        end
    else
        push!(state.infections, event)
    end
end

function execute!(state::SimState, params::SimParams, event::BecomeInfectiousEvent)
    @assert Infectious !== subjecthealth(state, event)
    @assert Recovered !== subjecthealth(state, event)
    state.infection_status[event.subject_id] = Infecitous
    
    add_potential_contacts!(HouseholdContact, state, params, event.subject_id)
    add_potential_contacts!(ConstantKernelContact, state, params, event.subject_id)
end

function execute!(state::SimState, params::SimParams, event::StayHomeEvent)
    if target_status == subjecthealth(state, event)
        state.infection_status[event.subject_id] = StayHome
    end
end

function execute!(state::SimState, params::SimParams, event::GoHospitalEvent)
    person_health = subjecthealth(state, event)
    if person_health == StayingHome || person_health == Infectious
        state.infection_status[event.subject_id] = Hospitalized
    end
end

function execute!(state::SimState, params::SimParams, event::RecoverEvent)
    @assert isactive(subjecthealth(event))
    
    state.infection_status[event.subject_id] = Recovered
end

function execute!(state::SimState, params::SimParams, event::DeathEvent)
    @assert Recovered !== subjecthealth(event)
    @assert Dead !== subjecthealth(event)

    state.infection_status[event.subject_id] = Dead
end

execute! (generic function with 6 methods)

In [38]:
function simulate!(state, params)
    while true
        if isempty(queue)
            @info "Empty queue"
            break
        end
        
        event = pop!(state.event_queue)
        if state.affected_people >= params.stop_simulation_threshold
            @info "The outbreak reached a high number $(params.stop_simulation_threshold)"
            break
        else
            event.time >= params.max_time
            @info "Max time reached"
            break
        end
        
        state.global_time = event.time
        
        execute!(state, params, event)
    end
end

| name |      size | summary            |
|:---- | ---------:|:------------------ |
| Base |           | Module             |
| Core |           | Module             |
| Main |           | Module             |
| df   | 3.111 MiB | 636307×3 DataFrame |


In [11]:

h = DataStructures.BinaryHeap{AbstractEvent, DataStructures.LessThan}()
push!(h, OutsideInfectionEvent(1.2, 34))
push!(h, OutsideInfectionEvent(1.3, 45))
push!(h, TransmissionEvent(1.25, 100, 56, ConstantKernelContact))

pop!(h) |> display
pop!(h) |> display
pop!(h) |> display

OutsideInfectionEvent(1.2f0, 34)

TransmissionEvent(1.25f0, 100, 56, ConstantKernelContact)

OutsideInfectionEvent(1.3f0, 45)